In [ ]:
# TEMP: Import lagom
# Not useful once lagom is installed
import sys
sys.path.append('/home/zuo/Code/lagom/')

# 1. Generate data

In [6]:
import random
import numpy as np

from car_racing import CarRacing


def make_env(seed=None, render_mode=False):
    env = CarRacing()
    if seed is not None:
        env.seed(seed)
        
    return env

def process_action(t, current_action):
    if t < 60:
        return np.array([0,1,0])
    
    if t % 5 > 0:
        return current_action

    rn = random.randint(0,9)
    if rn in [0]:
        return np.array([0,0,0])
    if rn in [1,2,3,4]:
        return np.array([0,random.random(),0])
    if rn in [5,6,7]:
        return np.array([-random.random(),0,0])
    if rn in [8]:
        return np.array([random.random(),0,0])
    if rn in [9]:
        return np.array([0,0,random.random()])

def adjust_obs(obs):
    return obs.astype('float32') / 255.

def generate_data(N, T, batch_size):
    assert N % batch_size == 0
    
    env = make_env()
    
    for i in range(N//batch_size):
        data = []

        for j in range(batch_size):
            epi_obs = []
            epi_action = []

            obs = adjust_obs(env.reset())
            a = env.action_space.sample()

            for t in range(T):
                a = process_action(t, a)

                epi_obs.append(obs)
                epi_action.append(a)

                observation, reward, done, info = env.step(a)
                obs = adjust_obs(observation)

                #env.render()

            data.append({'obs': epi_obs, 'action': epi_action})
            print(f'Batch {i}: collected {j}/{batch_size} episodes. ')
            

        np.save(f'logs/data_{i}', data)
        print(f'===> Finish collecting {(i+1)*batch_size}/{N} episodes')
    
    env.close()
            
generate_data(2000, 300, 200)


ModuleNotFoundError: No module named 'Box2D'